In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import math, time
import datetime
import numpy
import random
import keras
from sklearn.utils import shuffle
from keras.layers import LeakyReLU
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.layers.recurrent import LSTM
from keras.layers import Bidirectional
from keras.constraints import max_norm

# Multilayer Perceptron
from keras.utils import plot_model
from keras.models import Model
from keras.layers import Input
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers.merge import concatenate
# fixar random seed para se puder reproduzir os resultados

seed = 9
np.random.seed(seed)

Using TensorFlow backend.


In [2]:
def get_stock_data(stock_name, normalized=0,file_name=None):
    col_names = ['Month', 'Advertising', 'Sales', 'erro1', 'erro2', 'erro3', 'erro4']
    stocks = pd.read_csv("2advertising-and-sales-data-36-co.csv",sep=",", header=0,names=col_names)  # fica numa especie de tabela exactamente como estava no csv (1350 linhas,7 colunas)
    df = pd.DataFrame(stocks)  # neste caso não vai fazer nada
    date_split = df['Month'].str.split('-').str  # não vai servir para nada
    df['Ano'], df['Month'] = date_split  # não vai servir para nada
    df.drop(df.columns[[3, 4, 5, 6, 7]], axis=1, inplace=True)  # vou só ficar com as colunas 1 e 2 TAMBÉM PODEMOS QUERER A 0
    df = df.drop(df.index[[36, 37]])
    return df

In [3]:
def load_GOOGL_stock_dataset():
    stock_name = 'advertising'
    return get_stock_data(stock_name,0,"C:/Users/Elisa Valente/Desktop/4º ano/2semestre/Sistemas Autónomos/ComputerVision/2advertising-and-sales-data-36-co.csv")


In [4]:
def pre_processar_GOOGL_stock_dataset(df):
    df['Advertising'] = df['Advertising']
    df['Sales'] = df['Sales']

    conversion = {
        'Jan': 1,
        'Feb': 2,
        'Mar': 3,
        'Apr': 4,
        'May': 5,
        'Jun': 6,
        'Jul': 7,
        'Aug': 8,
        'Sep': 9,
        'Oct': 10,
        'Nov': 11,
        'Dec': 12,
    }


    # Categorizar meses to_categorize
    # acrescnetar estações do ano
    # categorizar dias com significado crescente

    meses = []
    for line in df['Month']:
        meses.append(conversion[line])
    df['Month'] = meses
    df = pd.get_dummies(df, columns=['Month'])

    estacoes = [4, 4, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3, 0, 0, 0, 1, 1, 1, 2, 2, 2, 3, 3, 3,
                4]
    df['Estacoes'] = estacoes
    df = pd.get_dummies(df, columns=['Estacoes'])

    ano = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2]

    #df['Ano'] = ano
    #df = pd.get_dummies(df, columns=['Ano'])
    #print(df)

    dias = [31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31, 30, 31, 30, 31, 31, 30, 31, 30, 31, 31, 28, 31,
            30, 31, 30, 31, 31, 30, 31, 30, 31]
    #df['Dias'] = dias
    #df['Dias'] = df['Dias']/10

    col_list = list(df)
    col_list[1], col_list[18] = col_list[18], col_list[1]
    df= df.ix[:, col_list]

    return df

In [ ]:
def load_data(df_dados, janela):
    qt_atributos = len(df_dados.columns)
    mat_dados = df_dados.as_matrix()#converter dataframe para matriz (lista com lista de cada registo)
    #print(mat_dados)
    tam_sequencia = janela + 1
    res = []
    for i in range(len(mat_dados) - tam_sequencia): #numero de registos - tamanho da sequencia
        res.append(mat_dados[i: i + tam_sequencia])
    res = np.array(res) #dá como resultado um np com uma lista de matrizes (janela deslizante ao longo da serie)
    qt_casos_treino = int(round(0.667 * res.shape[0])) #2/3% passam a ser casos de treino
    #print(qt_casos_treino, res.shape[0])
    train = res[:qt_casos_treino, :]
    x_train = train[:, :-1] #menos um registo pois o ultimo registo é o registo a seguir à janela
    y_train = train[:, -1][:,-1] #para ir buscar o último atributo para a lista dos labels
    x_test = res[qt_casos_treino:, :-1]
    y_test = res[qt_casos_treino:, -1][:,-1]
    x_train = np.reshape(x_train, (x_train.shape[0], x_train.shape[1], qt_atributos))
    x_test = np.reshape(x_test, (x_test.shape[0], x_test.shape[1], qt_atributos))
    x_train, y_train = shuffle(x_train, y_train)
    x_test, y_test = shuffle(x_test, y_test)
    #print(x_train)
    return [x_train, y_train, x_test, y_test]


In [ ]:
def build_model(janela,dropout1,dropout2,dropout3,lstm_nodes1,lstm_nodes2,dense_nodes,relu_alpha):
    model = Sequential()
    model.add(Bidirectional(LSTM(int(2**lstm_nodes1), activation='relu',input_shape=(janela, 19), return_sequences=True,kernel_constraint=max_norm(3), recurrent_constraint=max_norm(3), bias_constraint=max_norm(3))))
    model.add(Dropout(dropout1))
    model.add(LSTM(int(2**lstm_nodes2), activation='relu', return_sequences=False,kernel_constraint=max_norm(3), recurrent_constraint=max_norm(3), bias_constraint=max_norm(3)))
    model.add(Dropout(dropout2))
    #model.add(Dense(125, activation='relu'))
    model.add(Dense(int(2**dense_nodes)))
    model.add(LeakyReLU(alpha=relu_alpha))
    model.add(Dropout(dropout3))
    model.add(Dense(1, activation="linear", kernel_initializer="uniform"))
    model.compile(loss='mse', optimizer='adam', metrics=['accuracy'])

    return model

In [ ]:
def fitnessFunction(model, b_size, e_pochs):
    df = load_GOOGL_stock_dataset()
    df = pre_processar_GOOGL_stock_dataset(df)
    #print("df", df.shape)
    #print("tamanho do dataset", len(df))
    janela = 3 #tamanho da Janela deslizante 22
    X_train, y_train, X_test, y_test = load_data(df, janela)# o df[::-1] é o df por ordem inversa
    #print("X_train", X_train.shape)
    #print("y_train", y_train.shape)
    #print("X_test", X_test.shape)
    #print("y_test", y_test.shape)
    #model = build_model(janela)
    #model.fit(X_train, y_train, batch_size=512, epochs=2000, validation_split=0.1,verbose=0)
    history = model.fit(X_train, y_train, batch_size = int(b_size),epochs=int(e_pochs), validation_split= 0.01,verbose=0)
    #print_history_loss(history)
    #print_model(model,"lstm_model.png")
    trainScore = model.evaluate(X_train, y_train, verbose=0)
    #print('Train Score: %.2f MSE (%.2f RMSE)' % (trainScore[0], math.sqrt(trainScore[0])))
    testScore = model.evaluate(X_test, y_test, verbose=0)
    #print('Test Score: %.2f MSE (%.2f RMSE)' % (testScore[0], math.sqrt(testScore[0])))
    #print(model.metrics_names)
    #p = model.predict(X_test)
    #predic = np.squeeze(np.asarray(p)) #para transformar uma matriz de uma coluna e n linhas em um np array de n elementos
    #print_series_prediction(y_test,predic)
    return math.sqrt(testScore[0])

In [ ]:
def pop_fitness(janela,new_population, pop_size):
    scores = []
    for i in range(pop_size):
        model = build_model(janela,new_population[i][0],new_population[i][1],new_population[i][2],new_population[i][3],new_population[i][4],new_population[i][5],new_population[i][6])
        score = fitnessFunction(model,new_population[i][7],new_population[i][8])
        print(score)
        scores.append(score)
    return scores

In [ ]:
def select_mating_pool(population, scores, num_parents):
    parents = numpy.empty((num_parents, len(population[0])))
    for parent_num in range(num_parents):
        min_fitness_idx = numpy.where(scores == numpy.min(scores))
        min_fitness_idx = min_fitness_idx[0][0]
        parents[parent_num,:] = population[min_fitness_idx,:]
        scores[min_fitness_idx] = 1000
    return parents

In [ ]:
def crossover(parents, offspring_size):
    offspring = numpy.empty(offspring_size)
    # The point at which crossover takes place between two parents. Usually it is at the center.
    crossover_point = numpy.uint8(offspring_size[1]/2)
    for k in range(offspring_size[0]):
         # Index of the first parent to mate.
        parent1_idx = k%parents.shape[0]
        # Index of the second parent to mate.
        parent2_idx = (k+1)%parents.shape[0]
        # The new offspring will have its first half of its genes taken from the first parent.
        offspring[k, 0:crossover_point] = parents[parent1_idx, 0:crossover_point]
        # The new offspring will have its second half of its genes taken from the second parent.
        offspring[k, crossover_point:] = parents[parent2_idx, crossover_point:]
    return offspring

In [ ]:

def mutation(offspring_crossover):
    # Mutation changes a single gene in each offspring randomly.
    idx = numpy.random.randint(0, len(offspring_crossover))
    # The random value to be added to the gene.
    gene_number = numpy.random.randint(0, 6, 1)
    if (gene_number == 0):
        offspring_crossover[idx,0] = random.uniform(0, 1)
    elif (gene_number == 1):
        offspring_crossover[idx,1] = random.uniform(0, 1)
    elif (gene_number == 2):
        offspring_crossover[idx,2] = random.uniform(0, 1)
    elif (gene_number == 3):
        offspring_crossover[idx,3] = random.randint(1,7)
    elif (gene_number == 4):
        offspring_crossover[idx,4] = random.randint(1, 7)
    elif (gene_number == 5):
        offspring_crossover[idx,5] = random.randint(1, 7)
    elif (gene_number == 6):
        offspring_crossover[idx,5] = random.uniform(0, 1)
    elif (gene_number == 7):
        offspring_crossover[idx,5] = random.randint(0, 20)
    elif (gene_number == 7):
        offspring_crossover[idx,5] = random.randint(300, 3000)
    return offspring_crossover

In [ ]:
pop_size = 8
genes_number = 9 # Numero de genes
gen_numbers = 10  # Numero de geracoes
num_parents = int(pop_size/2)
offspring_number = int(pop_size/2)
janela=3

offspring_size = (offspring_number, genes_number)
new_population = []

for i in range(0,pop_size):
    n_lstms = random.randint(1, 1)
    pop = [random.uniform(0, 1),random.uniform(0, 1),random.uniform(0, 1),random.randint(1,7),random.randint(1, 7),random.randint(1, 7),random.uniform(0, 1), random.randint(0, 20),random.randint(300, 3000)]
    new_population.append(pop)
    
new_population = np.array(new_population)

for generation in range(gen_numbers):
    if(generation!=(gen_numbers-1)):
        print(new_population)
        scores = pop_fitness(janela, new_population, pop_size)
        print('Scores:' )
        print(scores)
        parents = select_mating_pool(new_population, scores, num_parents)
        #print('Parents:')
        #print(parents)
        offspring = crossover(parents, offspring_size)
        offspring_mutation = mutation(offspring)
        #print('Offspring:')
        #print(offspring)

        new_population[0:parents.shape[0]] = parents
        new_population[parents.shape[0]:pop_size] = offspring_mutation
    else:
        print(new_population)
        scores = pop_fitness(janela, new_population, pop_size)
        print('Scores:' )
        print(scores)

    
print("Finished!")   

[[3.12533915e-01 8.53768681e-01 2.36161342e-01 7.00000000e+00
  5.00000000e+00 6.00000000e+00 9.69331658e-01 1.10000000e+01
  2.27400000e+03]
 [9.96996703e-01 2.98236063e-01 9.67051030e-01 3.00000000e+00
  7.00000000e+00 4.00000000e+00 6.56641492e-01 1.10000000e+01
  4.72000000e+02]
 [7.86904138e-01 3.73382971e-01 9.55348618e-01 4.00000000e+00
  4.00000000e+00 5.00000000e+00 4.79733987e-01 8.00000000e+00
  2.64500000e+03]
 [6.59740154e-01 6.74416979e-01 3.92954360e-01 2.00000000e+00
  4.00000000e+00 7.00000000e+00 8.28764215e-01 2.00000000e+01
  1.85200000e+03]
 [1.33600376e-01 1.01033306e-01 5.48749686e-01 3.00000000e+00
  6.00000000e+00 2.00000000e+00 5.97654632e-03 1.90000000e+01
  2.43600000e+03]
 [4.42066956e-01 3.61864216e-01 3.49407318e-01 5.00000000e+00
  2.00000000e+00 2.00000000e+00 9.17917964e-02 5.00000000e+00
  1.40600000e+03]
 [1.90406401e-01 6.64643982e-01 8.53038632e-01 5.00000000e+00
  1.00000000e+00 6.00000000e+00 2.02550989e-01 1.60000000e+01
  6.80000000e+02]
 [8.35

C:\Users\Elisa Valente\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:49: DeprecationWarning: 
.ix is deprecated. Please use
.loc for label based indexing or
.iloc for positional indexing

See the documentation here:
http://pandas.pydata.org/pandas-docs/stable/indexing.html#ix-indexer-is-deprecated
C:\Users\Elisa Valente\AppData\Local\Programs\Python\Python36\lib\site-packages\ipykernel_launcher.py:3: FutureWarning: Method .as_matrix will be removed in a future version. Use .values instead.
  This is separate from the ipykernel package so we can avoid doing imports until


Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:
Use tf.cast instead.
0.18595882662112878
0.20738490506178262
0.180291518588924
0.17733416720476006
0.1819351945643863
0.17621845868616434
0.1931120610609368
0.18485240471023587
Scores:
[0.18595882662112878, 0.20738490506178262, 0.180291518588924, 0.17733416720476006, 0.1819351945643863, 0.17621845868616434, 0.1931120610609368, 0.18485240471023587]
[[4.42066956e-01 3.61864216e-01 3.49407318e-01 5.00000000e+00
  2.00000000e+00 2.00000000e+00 9.17917964e-02 5.00000000e+00
  1.40600000e+03]
 [6.59740154e-01 6.74416979e-01 3.92954360e-01 2.00000000e+00
  4.00000000e+00 7.00000000e+00 8.28764215e-01 2.00000000e+01
  1.85200000e+03]
 [7.86904138e-01 3.73382971e-01 9.55348618e-01 4.00000000e+00
  4.00000000e+00 5.00000000e+00 4.79733987e-01 8.00000000e+00
  2.64500000e+03]
 [1.33600376e-01 1.01033306e-01 5.48749686e-01 3.00000000e+00
  6.00000000e+00 2.0